# Team: GPA 4.1
### Dave Arno, Animesh Danayak, Michael Harris, Dhwani Mehta



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp "/content/drive/Shareddrives/423 Drive/Kafka/.env" .

In [ ]:
!pip install -q elasticsearch_dsl
!pip install -q python-dotenv

In [ ]:
import os, json, time, re
import pandas as pd
from dotenv import load_dotenv
from elasticsearch_dsl import Search
from elasticsearch import Elasticsearch

load_dotenv();

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn import decomposition
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
from nltk import WordNetLemmatizer


from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer



In [ ]:
def elasticsearch_connect():
  bonsai = os.environ['BONSAI_URL']
  auth = re.search('https\:\/\/(.*)\@', bonsai).group(1).split(':')
  host = bonsai.replace('https://%s:%s@' % (auth[0], auth[1]), '')
  # optional port
  match = re.search('(:\d+)', host)
  if match:
    p = match.group(0)
    host = host.replace(p, '')
    port = int(p.split(':')[1])
  else:
    port=443
  # Connect to cluster over SSL using auth for best security:
  es_header = [{
  'host': host,
  'port': port,
  'use_ssl': True,
  'http_auth': (auth[0],auth[1])
  }]
  # Instantiate the new Elasticsearch connection:
  es = Elasticsearch(es_header)
  # Verify that Python can talk to Bonsai (optional):
  es.ping()
  return es

In [ ]:
def gather_data(topic):
  es = elasticsearch_connect()
  s = Search(using=es, index=topic)
  df = pd.DataFrame([hit.to_dict() for hit in s.scan()])
  hits = [hit.to_dict() for hit in s.scan()]
  corpus = []
  for hit in hits:
    if hit['lang'] == 'en':
      if 'retweeted_status' in hit:
        if 'extended_tweet' in hit['retweeted_status']:
          corpus.append(hit['retweeted_status']['extended_tweet']['full_text'])
        else:
          corpus.append(hit['retweeted_status']['text'])
      elif 'extended_tweet' in hit:
        corpus.append(hit['extended_tweet']['full_text'])
      else:
        corpus.append(hit['text'])
  return corpus

**7.Twitter Text Analytics
Perform basic text-mining on “Important Tweets” to understand what
topics are trending on twitter.**


In [ ]:
vaccination_tweets =  "vaccination_tweets"
important_tweets = "important_tweets"
covid_tweets = "covid_tweets"
imp_tweets = pd.DataFrame(gather_data(important_tweets))

In [ ]:
imp_tweets.columns = ['tweets']
imp_tweets.head()

,tweets
0,"@yhaa_sark’s Nkuto\n\nFor Stronger,moisturized..."
1,"@us_ignorance Look, Russia is fighting in Syri..."
2,Congratulations DA💕
3,Dear @joelockhart: Just because the @GOP wants...
4,@unplasticday @SDG2030 @ConEdison @ConEdCEB @A...


**Remove special characters and lower case letters**

In [ ]:
def clean_text(string_in):
    string_in = re.sub("[^a-zA-Z]", " ", str(string_in))  # Replace all non-letters with spaces
    string_in = string_in.lower()                         # Tranform to lower case    
    
    return string_in.strip()

imp_tweets["tweets_cleaned"] = imp_tweets.tweets.apply(clean_text)
imp_tweets.head()

,tweets,tweets_cleaned
0,"@yhaa_sark’s Nkuto\n\nFor Stronger,moisturized...",yhaa sark s nkuto for stronger moisturized so...
1,"@us_ignorance Look, Russia is fighting in Syri...",us ignorance look russia is fighting in syria...
2,Congratulations DA💕,congratulations da
3,Dear @joelockhart: Just because the @GOP wants...,dear joelockhart just because the gop wants...
4,@unplasticday @SDG2030 @ConEdison @ConEdCEB @A...,unplasticday sdg conedison conedceb ai...


**Tokenization of the text**

In [ ]:
preprocessed = [" ".join(RegexpTokenizer(r'\w+').tokenize(imp_tweets.tweets_cleaned[idx])) for idx in imp_tweets.index]
preprocessed

['yhaa sark s nkuto for stronger moisturized soft and shiny hair growth good for all hair types locks itchy scalp relief and dandruff very affordable dm yhaa sark and get yours now ingredients aloevera jojoba castor oil tea tree peppermint fragrance https t co mbehnwupf',
 'us ignorance look russia is fighting in syria but the us just grabbed oil and is sitting on it all financial markets belong to them the dollar accounts for of all currencies their sanctions are based on this',
 'congratulations da',
 'dear joelockhart just because the gop wants to deprive voters of water wants to cancel baseball and embraces a lawyer who said hugo chavez rose from the dead to rig the election does not make the gop an unserious party wait what am i saying ok maybe you re correct https t co pq saubilf',
 'unplasticday sdg conedison conedceb aidanrgallagher pamelafalk seforallorg bsebti elonmusk razarumi poet economist joselyn dumas carolndosi paulpolman billpeduto oecd local uclg org vincentcoyle brit

**Lemmatization**

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
  return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
imp_tweets["text_lemmatized"] = imp_tweets.tweets.apply(lemmatize_text)

print(imp_tweets["text_lemmatized"])

0       [@yhaa_sark’s, Nkuto, For, Stronger,moisturize...
1       [@us_ignorance, Look,, Russia, is, fighting, i...
2                                  [Congratulations, DA💕]
3       [Dear, @joelockhart:, Just, because, the, @GOP...
4       [@unplasticday, @SDG2030, @ConEdison, @ConEdCE...
                              ...                        
2014    [Then,, you, still, the, brain-dead, climate, ...
2015    [@KingNeverlies, The, others, work., But, that...
2016    [A, reminder, that, @amnesty's, microsite, "Te...
2017    [Ocean, action, is, #ClimateAction!, A, new, U...
2018    [A, year, after, the, start, of, COVID,, Alber...
Name: text_lemmatized, Length: 2019, dtype: object


**Removing Stop words**

In [ ]:
# As defined in the scikit-learn package
custom_stop_words = ['gziqqbttdc','syfnu', 'qxbdqu', 'fufu', 'yriohq', 'uqmmznnedt', 'https', 'oott', 'khasra', 'tf','khurd']
my_stop_words = text.ENGLISH_STOP_WORDS.union(custom_stop_words)

**TF-IDF**

In [ ]:
vectorizer = TfidfVectorizer(min_df = 1, ngram_range = (1,1), 
                             stop_words = my_stop_words)

tfidf = vectorizer.fit_transform(preprocessed)
print("Created document-term matrix of size %d x %d" % (tfidf.shape[0],tfidf.shape[1]))

Created document-term matrix of size 2019 x 9628


Non-Negative matrix factorization



In [ ]:
nmf = decomposition.NMF(init = 'nndsvd', n_components = 5, max_iter = 200)
W = nmf.fit_transform(tfidf)
H = nmf.components_
print("Generated W(document-topic)) matrix of size %s and H (topic-word) matrix of size %s" % ( str(W.shape), str(H.shape)))

feature_names = vectorizer.get_feature_names()
n_top_words = 10

# Print top words in each topic
for topic_idx, topic in enumerate(H):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Generated W(document-topic)) matrix of size (2019, 5) and H (topic-word) matrix of size (5, 9628)
Topic #0:
heating amp refused loyal contract taken facebook fired hours qg

Topic #1:
supposed served perfume oil paintings earrings painting cbd rig count

Topic #2:
kerry invested companies biden papers spirals joining czar admin climate

Topic #3:
amp facial cream super skin relentless skincareproducts acceptance hydrated leave

Topic #4:
delhi gas people oxygen verified oil uk let ukparliament biafra



**Report and analyse the top words from TF-IDF matrix**

The TF-IDF gives us the frequecy of each word in a document, a numnber thats proportional to its frequency in the document and inversely proportional to number of documents in which it appears. Based on our top words, there is no obvious pattern here, beyond the fact that oil, climate, gas and heating seem to characterize the majority of articles. What might be more interesting, is to separately consider groups of documents falling into a particular category.

## Q7 ii)
At a high level, we believe that the TFIDF matrix is an effective indicator of the important topic. However, more data cleaning and noise reduction would be necessary to ensure that we're only picking up the important words; as it stands now, some of the outputs from the TFIDF matrix are not accurate indicators for our important topic.

It would be worthwhile to explore changing our search terms within the topic, to see if this is an effective mechanism to eliminate some noise. Moreover, our team only scraped several thousand tweets; it would be more accurate to scrape hundreds of thousands of tweets to more accurately gauge the topic.

Moreover, it may be necessary to conduct sentiment analysis to isolate perception towards this important topic. Otherwise, we'll pick up lots of keywords for which it is difficult to make specific inferences.

## **8.Twitter Sentiment Analytics**

# Vaccination Tweets

In [ ]:
vacc_tweets = pd.DataFrame(gather_data(vaccination_tweets))
vacc_tweets.columns = ['tweets']
vacc_tweets.head()

,tweets
0,#ApolloHospitals will begin #vaccination for t...
1,Fantastic reporting by @ArunDev1 on people who...
2,@TrumpFrederick @secupp Im not talking about t...
3,India’s surge is as a result of the vaccine !!...
4,People are having vaccination parties to celeb...


In [ ]:
def clean_text(string_in):
    string_in = re.sub("[^a-zA-Z]", " ", str(string_in))  # Replace all non-letters with spaces
    string_in = string_in.lower()                         # Tranform to lower case    
    
    return string_in.strip()

vacc_tweets["tweets_cleaned"] = vacc_tweets.tweets.apply(clean_text)
vacc_tweets.head()

,tweets,tweets_cleaned
0,#ApolloHospitals will begin #vaccination for t...,apollohospitals will begin vaccination for th...
1,Fantastic reporting by @ArunDev1 on people who...,fantastic reporting by arundev on people who...
2,@TrumpFrederick @secupp Im not talking about t...,trumpfrederick secupp im not talking about th...
3,India’s surge is as a result of the vaccine !!...,india s surge is as a result of the vaccine ...
4,People are having vaccination parties to celeb...,people are having vaccination parties to celeb...


In [ ]:
preprocessed = [" ".join(RegexpTokenizer(r'\w+').tokenize(vacc_tweets.tweets_cleaned[idx])) for idx in vacc_tweets.index]

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
  return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
vacc_tweets["text_lemmatized"] = vacc_tweets.tweets.apply(lemmatize_text)

print(vacc_tweets["text_lemmatized"])

0      [#ApolloHospitals, will, begin, #vaccination, ...
1      [Fantastic, reporting, by, @ArunDev1, on, peop...
2      [@TrumpFrederick, @secupp, Im, not, talking, a...
3      [India’s, surge, is, a, a, result, of, the, va...
4      [People, are, having, vaccination, party, to, ...
                             ...                        
899    [Vaccination, passport, will, be, the, ditch, ...
900    [⚠️, A, Very, Serious, Concern,, Folks, ⚠️, 💉,...
901    [Live, music, at, Las, Vegas, vaccination, sit...
902    [🏴󠁧󠁢󠁳󠁣󠁴󠁿, As, Scotland, completes, coronavirus...
903    [My, friend, Mum, in, Hyderabad, need, medicin...
Name: text_lemmatized, Length: 904, dtype: object


In [ ]:
# As defined in the scikit-learn package
custom_stop_words = ['gziqqbttdc','syfnu', 'qxbdqu', 'fufu', 'yriohq', 'uqmmznnedt', 'https', 'oott', 'khasra', 'tf','khurd']
my_stop_words = text.ENGLISH_STOP_WORDS.union(custom_stop_words)

In [ ]:
vectorizer = TfidfVectorizer(min_df = 1, ngram_range = (1,1), 
                             stop_words = my_stop_words)

tfidf = vectorizer.fit_transform(preprocessed)
print("Created document-term matrix of size %d x %d" % (tfidf.shape[0],tfidf.shape[1]))

Created document-term matrix of size 904 x 4380


In [ ]:
nmf = decomposition.NMF(init = 'nndsvd', n_components = 5, max_iter = 200)
W = nmf.fit_transform(tfidf)
H = nmf.components_
print("Generated W(document-topic)) matrix of size %s and H (topic-word) matrix of size %s" % ( str(W.shape), str(H.shape)))

feature_names = vectorizer.get_feature_names()
n_top_words = 10

# Print top words in each topic
for topic_idx, topic in enumerate(H):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Generated W(document-topic)) matrix of size (904, 5) and H (topic-word) matrix of size (5, 4380)
Topic #0:
hyderabad thymosin mg covidhelphyderabad urgently alpha appreciated medicines friend medicine

Topic #1:
zwcqaxc yo watching usa vaccinate country age getting people covid

Topic #2:
selenagomez borisjohnson potus vp uk vaxlive onecampaign znoatd commit thanks

Topic #3:
fake people cards reflect frown immigrants documents undocumented families buying

Topic #4:
vaccine covid doctor kenyan dies anti dnuowtud bre biihuo karanja



# Covid Tweets

In [ ]:
cov_tweets = pd.DataFrame(gather_data(covid_tweets))
cov_tweets.columns = ['tweets']

cov_tweets["tweets_cleaned"] = cov_tweets.tweets.apply(clean_text)
cov_tweets.head()

,tweets,tweets_cleaned
0,"The @NatGeo documentary ""Mission Possible-The ...",the natgeo documentary mission possible the ...
1,After refusing to budge on PCR cycle threshold...,after refusing to budge on pcr cycle threshold...
2,"free /friː/\nadjective, adverb\n\ncosting noth...",free fri adjective adverb costing nothing...
3,@MobilePunch This headline should read Pfizer-...,mobilepunch this headline should read pfizer b...
4,"@bhrenton Given the date of campus departure, ...",bhrenton given the date of campus departure s...


In [ ]:
preprocessed = [" ".join(RegexpTokenizer(r'\w+').tokenize(cov_tweets.tweets_cleaned[idx])) for idx in cov_tweets.index]

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
  return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
cov_tweets["text_lemmatized"] = cov_tweets.tweets.apply(lemmatize_text)

print(cov_tweets["text_lemmatized"])

0       [The, @NatGeo, documentary, "Mission, Possible...
1       [After, refusing, to, budge, on, PCR, cycle, t...
2       [free, /friː/, adjective,, adverb, costing, no...
3       [@MobilePunch, This, headline, should, read, P...
4       [@bhrenton, Given, the, date, of, campus, depa...
                              ...                        
1325    [Second, Moderna, Shot!, In, two, week, I’ll, ...
1326    [Speaker, Jason, Wentworth, is, concerned, abo...
1327    [.@IBMSecurity, said, that,, in, 2020,, phishi...
1328    [Biden, won't, rule, out, requiring, U.S., ser...
1329    [If, you’re, 30-39, y/o, (non-priority),, you,...
Name: text_lemmatized, Length: 1330, dtype: object


In [ ]:
vectorizer = TfidfVectorizer(min_df = 1, ngram_range = (1,1), 
                             stop_words = my_stop_words)

tfidf = vectorizer.fit_transform(preprocessed)
print("Created document-term matrix of size %d x %d" % (tfidf.shape[0],tfidf.shape[1]))

Created document-term matrix of size 1330 x 4998


In [ ]:
# As defined in the scikit-learn package
custom_stop_words = ['gziqqbttdc','syfnu', 'qxbdqu', 'fufu', 'yriohq', 'uqmmznnedt', 'https', 'oott', 'khasra', 'tf','khurd','zwcqaxc ','vxpgkup','vaers']
my_stop_words = text.ENGLISH_STOP_WORDS.union(custom_stop_words)

In [ ]:
nmf = decomposition.NMF(init = 'nndsvd', n_components = 5, max_iter = 200)
W = nmf.fit_transform(tfidf)
H = nmf.components_
print("Generated W(document-topic)) matrix of size %s and H (topic-word) matrix of size %s" % ( str(W.shape), str(H.shape)))

feature_names = vectorizer.get_feature_names()
n_top_words = 10

# Print top words in each topic
for topic_idx, topic in enumerate(H):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Generated W(document-topic)) matrix of size (1330, 5) and H (topic-word) matrix of size (5, 4998)
Topic #0:
zwcqaxc watching yo usa vaccinate age country getting people covid

Topic #1:
vaers reports week backlog database moly rise holy federal updated

Topic #2:
people warning remain cautious nv vxpgkup admitted figures suggested researchers

Topic #3:
view drive continue centers open need vaccine covid produce cares

Topic #4:
vaccine covid pfizer dose moderna amp doses india vaccines nd



**Sentiment Analysis for COVID-19 Vaccination tweets**

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
total_score=0

tweets = cov_tweets["tweets_cleaned"]
sentiment = SentimentIntensityAnalyzer()
for sentence in tweets:						
  ss = sentiment.polarity_scores(sentence)						
  total_score -= ss["neg"]						
  total_score += ss["pos"]										
  total_score = round(total_score,3)										

total_score

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


33.533

**Sentiment Analysis for Vaccination Tweets**

In [ ]:
total_score=0

tweets = vacc_tweets["tweets_cleaned"]
sentiment = SentimentIntensityAnalyzer()
for sentence in tweets:						
  ss = sentiment.polarity_scores(sentence)						
  total_score -= ss["neg"]						
  total_score += ss["pos"]										
  total_score = round(total_score,3)										

total_score

20.835

**Analyze the difference between the sentiment of the public towards the
vaccination as a whole versus the COVID-19 vaccination**

While both Vaccination and COVID-19 Vaccination have positive overall sentiments, COVID-19 Vaccination tweets have a higher positive sentiment as opposed to Vaccination tweets. The two corpuses have overlapping topics, and in many cases, it was hard to differentiate sentiment towards vaccinations (most of which are COVID-related right now) and COVID-19 in general.